In [203]:
# img_path='./custom-data/lol-2.jpg'
# image = Image.open(img_path).convert('L')
# image.show()
# image = ToTensor()(image) # unsqueeze to add artificial first dimension
# # image = transforms.ToPILImage(image)
# # image.convert("L")
# # image = ToTensor()(image)

# print(image.size())
# print(image)



import itertools

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from IPython.display import Image
from IPython import display
import matplotlib.pyplot as plt

%matplotlib inline

from PIL import Image
from torchvision.transforms import ToTensor

img_path2 = './test2.png'
image2 = Image.open(img_path2).convert('L')
image2 = ToTensor()(image2)

orig_height = image2.size()[1]
orig_width =image2.size()[2]
# print(image2)

print(image2)

torch_dic = {}
torch_width_dic = {}
dic_num = 0
startX = 0
endX = 0
startflag = 0
black = 0

for i in range(orig_width):                                         ## 1 이면 white  , 0 이면 black 
    black = 0                      
    for j in range(orig_height):  # 세로로 scan
        
        
        if(startflag == 0 ): # 흰색을 지나는 중일때 
            if(image2[0][j][i] == 0):   # 검정색을 발견하면 
                startflag = 1
                startX = i - 5


        if(startflag == 1):  # 검정을 지나는 중일때
            if(image2[0][j][i] ==  0):     # 검정을 발견하면 black = 1 검정이 아에 없으면 black= 0
                black = 1 
            
    # 세로로 scan 끝내고          
        
    if(  startflag == 1  and  black ==  0 ):       # 검정을 지나다가 검정을 아에 발견 못함 > 하나의 덩어리 끝 
        endX = i + 5
        startflag = 0
        black = 0
    
        temp = torch.rand(1,orig_height,endX-startX)
        for k in range(0,endX-startX):            # temp tensor 만듬. 
            for i in range(orig_height):
                temp[0][i][k] = image2[0][i][k+startX]
        dic_num = dic_num + 1
        torch_dic[str(dic_num)] = temp
        torch_width_dic[str(dic_num)] = endX-startX
        startX = 0
        endX = 0
        
final_dic = {}
torch_height_dic = {}
startY = 0
endY = 0
startflag = 0
final_num = 0


for i in range (dic_num):   

    i = i + 1         # 0~ dic_num -1  이니까 +1 
    image = torch_dic[str(i)]
    width = torch_width_dic[str(i)]
    
  
    for j in range(orig_height): 
        # 가로로 스캔 
        black = 0
        for k in range (width): 
            
            if(startflag == 0 ): # 흰색을 지나는 중일때 
                if(image[0][j][k] == 0):   # 검정색을 발견하면 
                    startflag = 1
                    startY = j - 5


            if(startflag == 1):  # 검정을 지나는 중일때
                if(image[0][j][k] ==  0):     # 검정을 발견하면 black = 1 검정이 아에 없으면 black= 0
                    black = 1 
                    
        # 가로 스캔 끝내고                 
        if(  startflag == 1  and  black ==  0 ):       # 검정을 지나다가 검정을 아에 발견 못함 > 하나의 덩어리 끝 
            endY = j + 5
            startflag = 0
            black = 0

            temp = torch.rand(1,endY-startY,width)
            for i in range(0,endY-startY):            # temp tensor 만듬. 
                for j in range(width):
                    temp[0][i][j] = image[0][i+startY][j]
            final_num = final_num+ 1
            final_dic[str(final_num)] = temp
            torch_height_dic[str(dic_num)] = endY-startY
            startY = 0
            endY = 0
            


    


tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [204]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, 
                          nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))       # variable로 만들고 
            
        # One time step
        out, hn = self.rnn(x, h0)       # rnn 한번 
        out = self.fc(out[:, -1, :]) 
        return out


In [205]:
print(len(final_dic)+1)
model = RNNModel(28,100,3,19)
model.load_state_dict(torch.load("./models/final2"))
model.eval()

16


RNNModel(
  (rnn): RNN(28, 100, num_layers=3, batch_first=True)
  (fc): Linear(in_features=100, out_features=19, bias=True)
)

In [206]:
def make_square(im, fill_color=(255, 255, 255, 255)):
    x, y = im.size
    size = max(x, y)
    new_im = Image.new('RGBA', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

In [207]:
from torch.autograd import Variable

char_dic = {'0':'0','1':'1','2':'2','3':'3','4':'4',
            '5':'5','6':'6','7':'7','8':'8','9':'9',
            '10':'+','11':'-','12':'(','13':')','14':'[',
            '15':']', '16':'x', '17':',','18' : '.'}

result_dic = []
print(len(final_dic)+1)
for i in range(1, len(final_dic)+1):
    
    result = final_dic[str(i)]
    result = transforms.ToPILImage()(result) 
    result = make_square(result,fill_color=(255, 255, 255, 255))
    result = result.resize((28,28),Image.ANTIALIAS)
#     result.show()
    result = ToTensor()(result)
    
    
    output_test = model(result)
    predicted = torch.max(output_test.data, 1)[1]
    
    
    
    result_dic.insert(i-1,char_dic[str(int(predicted[0]))]) 
print(result_dic)

16
['(', '6', 'x', '8', 'x', '2', ')', '+', '(', '1', '1', '4', '1', '0', ')']


In [210]:

import numpy as np
def vector_calculate (vec1, vec2, operator):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    if operator == '+':
        if (len(vec1) != len(vec2)):
            raise SyntaxError
        return vec1 + vec2
    elif operator == '-':
        if (len(vec1) != len(vec2)):
            raise SyntaxError
        return vec1 - vec2
    elif operator == 'x':
        if (len(vec1) == 1 or len(vec2) == 1):
            return vec1 * vec2
        return np.cross(vec1, vec2)
    elif operator == '*':
        return np.dot(vec1, vec2)
    else:
        raise operatorError

def calculate (recog_list):
    for i in range(len(recog_list)):
        if recog_list[i] == ')' or recog_list[i] == ']':
            operand1 = recog_list[1:i]
            operator = recog_list[i + 1]
            operand2 = recog_list[i + 3 : len(recog_list) - 1]
            break
    operand1 = operand1[0:len(operand1):2]
    operand2 = operand2[0:len(operand2):2]
    for i in range(len(operand1)):
        operand1[i] = int(operand1[i])
    for i in range(len(operand2)):
        operand2[i] = int(operand2[i])
    return vector_calculate(operand1, operand2, operator)

In [211]:
calculate(result_dic)

array([ 7, 12,  2])